In [1]:
import re 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
import random 
import math
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
from tqdm import tqdm_notebook
from sklearn.preprocessing import MinMaxScaler
import torch.nn.functional as F
import d2l
import time
import traceback
import fastprogress
from torchmetrics.classification import BinaryAccuracy, Accuracy 

In [2]:
df = pd.read_csv("C:/Users/kacpe/Desktop/study/research lab/data_model_v2.csv")
# List of column names to drop
columns_to_drop = ['lKnee_x','lKnee_y','lKnee_z','lAnkle_x','lAnkle_y','lAnkle_z','rKnee_x','rKnee_y','rKnee_z','rAnkle_x','rAnkle_y','rAnkle_z']
df = df.drop(columns=columns_to_drop)
# Step 1: Separate 'id' and 'trial' columns from the rest of the data
data_to_scale = df.drop(columns=['id', 'trial'])

# Step 2: Apply MinMaxScaler to the remaining columns
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_to_scale)

# Convert the scaled data back to a DataFrame
scaled_df = pd.DataFrame(scaled_data, columns=data_to_scale.columns)

# Step 3: Merge 'id' and 'trial' columns with the scaled data
scaled_df[['id', 'trial']] = df[['id', 'trial']]

# Step 4: Split the data into training and test sets based on the 'trial' column
train_set = scaled_df[scaled_df['trial'].isin(range(1, 15))].drop(columns=['id', 'trial'])
test_set = scaled_df[scaled_df['trial']==15].drop(columns=['id', 'trial'])
val_set = scaled_df[scaled_df['trial']==16].drop(columns=['id', 'trial'])
full_set = scaled_df.drop(columns=['id','trial'])

# split data into x and y 
X_train, y_train = train_set.iloc[:,:42], train_set.iloc[:,42:]
X_test, y_test = test_set.iloc[:,:42], test_set.iloc[:,42:]
X_val, y_val = val_set.iloc[:,:42], val_set.iloc[:,42:]
X, y = full_set.iloc[:,:42], full_set.iloc[:,42:]

In [3]:
# Custom Dataset class for loading data
class MyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        # Implement data retrieval for each index
        input_data = self.X[idx]
        target_data = self.y[idx]
        input_data = input_data.unsqueeze(0)
        
        # Convert data to torch tensors if required
        input_tensor = torch.Tensor(input_data)
        target_tensor = torch.Tensor(target_data)
        
        return input_tensor, target_tensor

In [4]:
# Create custom datasets for training, validation, and testing
full_dataset = MyDataset(torch.tensor(X.values), torch.tensor(y.values))
train_dataset = MyDataset(torch.tensor(X_train.values), torch.tensor(y_train.values))
val_dataset = MyDataset(torch.tensor(X_val.values), torch.tensor(y_val.values))
test_dataset = MyDataset(torch.tensor(X_test.values), torch.tensor(y_test.values))

In [5]:
# Create a DataLoader
#batch_size = 5561#67  # Set your desired batch size
#shuffle = False  # Set to False to preserve the order of your data
fullset_dataloader = DataLoader(full_dataset, batch_size=X.shape[0], shuffle=False)
train_dataloader = DataLoader(train_dataset, batch_size=X_train.shape[0], shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=X_test.shape[0], shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=X_val.shape[0], shuffle=False)

In [6]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [7]:
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2, bidirectional=False):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        out = F.softmax(out, dim=1)
        return out, h
    
    #def init_hidden(self, batch_size):
        #weight = next(self.parameters()).data
        #hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        #return hidden
    def init_hidden(self, batch_size):
        if batch_size > 1:
            weight = next(self.parameters()).data
            hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        else:
            weight = next(self.parameters()).data
            hidden = weight.new(self.n_layers, self.hidden_dim).zero_().to(device)
        return hidden

In [8]:
class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(LSTMNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob, bidirectional=False)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.lstm(x, h)
        out = self.fc(self.relu(out[:,-1]))
        out = F.softmax(out, dim=1)
        return out, h 
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [9]:
input_shape = next(iter(train_dataloader))[0].shape
print("Input shape:", input_shape)

Input shape: torch.Size([6107, 1, 42])


In [10]:
def train(train_loader, learn_rate, hidden_dim=32, epochs=5, model_type="GRU", threshold=0.5, criterion=nn.BCEWithLogitsLoss(),    input_dim = 42, n_layers = 10):
    
    # Setting common hyperparameters
    input_dim = 42
    output_dim = 4
    n_layers = 10
    # Instantiating the models
    if model_type == "GRU":
        model = GRUNet(input_dim, hidden_dim, output_dim, n_layers)
    else:
        model = LSTMNet(input_dim, hidden_dim, output_dim, n_layers)
    model.to(device)
    
    # Defining loss function and optimizer
    #criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
    
    model.train()
    print("Starting Training of {} model".format(model_type))
    epoch_times = []
    train_losses = []
    
    # Start training loop
    for epoch in range(1,epochs+1):
        start_time = time.perf_counter()
        h = model.init_hidden(batch_size)
        avg_loss = 0.
        counter = 0
        total_correct = 0
        total_samples = 0  
        for x, label in train_loader:
            counter += 1
            if model_type == "GRU":
                h = h.data
            else:
                h = tuple([e.data for e in h])
            model.zero_grad()
            
            out, h = model(x.to(device).float(), h)
            predicted_labels = (out > threshold).float()
            #accuracy = Accuracy(predicted_labels, label.to(device).float())
            loss = criterion(out, label.to(device).float())
            loss.backward()
            optimizer.step()
            avg_loss += loss.item()
            #avg_acc = accuracy.mean()
            # Compute accuracy
            #predicted_labels = (out > threshold).long()
            total_correct += (predicted_labels == label.to(device)).sum().item()
            total_samples += label.size(0)
            train_losses.append(avg_loss)
            if counter%200 == 0:
                print("Epoch {}......Step: {}/{}....... Average Loss for Epoch: {}".format(epoch, counter, len(train_loader), avg_loss/counter))
        current_time = time.perf_counter()
            # Calculate accuracy metric
        
    
    
        current_time = time.perf_counter()
        #print("Epoch {}/{} Done, avg Loss: {}, Accuracy: {}".format(epoch, epochs, avg_loss, avg_acc.item()))
        print("Epoch {}/{} Done, avg Loss: {}".format(epoch, epochs, avg_loss))
        #print("Total Time Elapsed: {} seconds".format(str(current_time - start_time)))
        epoch_times.append(current_time - start_time)
        
        #print("Epoch {}/{} Done, Total Loss: {}".format(epoch, epochs, avg_loss/len(train_loader)))
        #print("Total Time Elapsed: {} seconds".format(str(current_time-start_time)))
        #epoch_times.append(current_time-start_time)
    print("Total Training Time: {} seconds".format(str(sum(epoch_times))))
    return model, train_losses 

def evaluate(model, test_x, test_y, label_scalers):
    model.eval()
    outputs = []
    targets = []
    start_time = time.perf_counter()
    for i in test_x.keys():
        inp = torch.from_numpy(np.array(test_x[i]))
        labs = torch.from_numpy(np.array(test_y[i]))
        h = model.init_hidden(inp.shape[0])
        out, h = model(inp.to(device).float(), h)
        outputs.append(label_scalers[i].inverse_transform(out.cpu().detach().numpy()).reshape(-1))
        targets.append(label_scalers[i].inverse_transform(labs.numpy()).reshape(-1))
    print("Evaluation Time: {}".format(str(time.perf_counter()-start_time)))
    sMAPE = 0
    for i in range(len(outputs)):
        sMAPE += np.mean(abs(outputs[i]-targets[i])/(targets[i]+outputs[i])/2)/len(outputs)
    print("sMAPE: {}%".format(sMAPE*100))
    return outputs, targets, sMAPE

In [11]:
X_train.shape

(6107, 42)

In [12]:
threshold = 0.5
type(threshold)

float

In [13]:
batch_size=X.shape[0]
criterion = nn.BCEWithLogitsLoss()
lr = 0.01
LSTM_model, losses = train(fullset_dataloader, lr, model_type="GRU",hidden_dim=24, epochs=1500, criterion = criterion)

Starting Training of GRU model
Epoch 1/1500 Done, avg Loss: 0.673486590385437
Epoch 2/1500 Done, avg Loss: 0.6721745133399963
Epoch 3/1500 Done, avg Loss: 0.6708976030349731
Epoch 4/1500 Done, avg Loss: 0.6694795489311218
Epoch 5/1500 Done, avg Loss: 0.667730450630188
Epoch 6/1500 Done, avg Loss: 0.6654906868934631
Epoch 7/1500 Done, avg Loss: 0.6624553799629211
Epoch 8/1500 Done, avg Loss: 0.6585700511932373
Epoch 9/1500 Done, avg Loss: 0.6538698673248291
Epoch 10/1500 Done, avg Loss: 0.6489112973213196
Epoch 11/1500 Done, avg Loss: 0.6443926095962524
Epoch 12/1500 Done, avg Loss: 0.6405400633811951
Epoch 13/1500 Done, avg Loss: 0.6374569535255432
Epoch 14/1500 Done, avg Loss: 0.6351756453514099
Epoch 15/1500 Done, avg Loss: 0.6336790323257446
Epoch 16/1500 Done, avg Loss: 0.6328139305114746
Epoch 17/1500 Done, avg Loss: 0.6323539018630981
Epoch 18/1500 Done, avg Loss: 0.6321682929992676
Epoch 19/1500 Done, avg Loss: 0.6320831775665283
Epoch 20/1500 Done, avg Loss: 0.6320508718490601


In [14]:
def accuracy(correct, total): 
    """Compute accuracy as percentage.

    Args:
        correct (int): Number of samples correctly predicted.
        total (int): Total number of samples

    Returns:
        float: Accuracy
    """
    return float(correct)/total

In [15]:
def evaluate(model, test_x, test_y):
    model.eval()
    outputs = []
    targets = []
    predictions = []  # Add this list to store the binary predictions

    start_time = time.perf_counter()
    for i in test_x.keys():
        inp = torch.from_numpy(np.array(test_x[i]))
        labs = torch.from_numpy(np.array(test_y[i]))
        h = model.init_hidden(inp.shape[0])
        out, h = model(inp.to(device).float(), h)
        #outputs.append(label_scalers[i].inverse_transform(out.cpu().detach().numpy()).reshape(-1))
        #targets.append(label_scalers[i].inverse_transform(labs.numpy()).reshape(-1))
        
        # Apply sigmoid and threshold to get binary predictions
        sigmoid_out = torch.sigmoid(out)
        threshold = 0.5  # You can adjust this threshold as needed
        binary_predictions = (sigmoid_out > threshold).long()
        predictions.append(binary_predictions.cpu().detach().numpy().reshape(-1))
        
    print("Evaluation Time: {}".format(str(time.perf_counter()-start_time)))
    sMAPE = 0
    for i in range(len(outputs)):
        sMAPE += np.mean(abs(outputs[i]-targets[i])/(targets[i]+outputs[i])/2)/len(outputs)
    print("sMAPE: {}%".format(sMAPE*100))
    return outputs, targets, sMAPE, predictions  # Return the binary predictions as well

In [16]:
X_train_dict = X_train.to_dict(orient='dict')
y_train_dict = y_train.to_dict(orient='dict')

In [17]:
def evaluate(model, test_loader, threshold=0.5):
    model.eval()
    outputs = []
    targets = []
    predictions = []
    
    with torch.no_grad():
        for x, label in test_loader:
            h = model.init_hidden(x.shape[0])
            out, h = model(x.to(device).float(), h)
            outputs.append(F.sigmoid(out).cpu().detach().numpy())  # Apply sigmoid
            targets.append(label.cpu().detach().numpy())
            threshold = 0.5  # You can adjust this threshold as needed
            sigmoid_out = torch.sigmoid(out)
            binary_predictions = (sigmoid_out > threshold).long()
            predictions.append(binary_predictions.cpu().detach().numpy().reshape(-1))
    
    accuracy = calculate_accuracy(outputs, targets, threshold)
    print("Accuracy: {:.2f}%".format(accuracy * 100))
    return outputs, targets, accuracy, predictions 

def calculate_accuracy(outputs, targets, threshold=0.5):
    total_samples = 0
    total_correct = 0
    
    for output, target in zip(outputs, targets):
        predicted_labels = (output > threshold).astype(int)
        total_correct += (predicted_labels == target).sum()
        total_samples += target.size
    
    accuracy = total_correct / total_samples
    return accuracy

In [18]:
outputs, targets, accuracy, predictions = evaluate(LSTM_model, test_dataloader)

Accuracy: 61.18%


c:\Users\kacpe\anaconda3\lib\site-packages\torch\nn\functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [19]:
outputs

[array([[0.5133507 , 0.72042674, 0.5000007 , 0.50000054],
        [0.5133507 , 0.72042674, 0.5000007 , 0.50000054],
        [0.5133507 , 0.72042674, 0.5000007 , 0.50000054],
        ...,
        [0.5133507 , 0.72042674, 0.5000007 , 0.50000054],
        [0.5133507 , 0.72042674, 0.5000007 , 0.50000054],
        [0.5133507 , 0.72042674, 0.5000007 , 0.50000054]], dtype=float32)]

In [20]:
outputs

[array([[0.5133507 , 0.72042674, 0.5000007 , 0.50000054],
        [0.5133507 , 0.72042674, 0.5000007 , 0.50000054],
        [0.5133507 , 0.72042674, 0.5000007 , 0.50000054],
        ...,
        [0.5133507 , 0.72042674, 0.5000007 , 0.50000054],
        [0.5133507 , 0.72042674, 0.5000007 , 0.50000054],
        [0.5133507 , 0.72042674, 0.5000007 , 0.50000054]], dtype=float32)]

In [21]:
targets

[array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        ...,
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])]

In [22]:
batch_size = X.shape[0]
criterion = nn.BCEWithLogitsLoss()
lr = 0.001
#LSTM_model, losses_LSTM_1000 = train(fullset_dataloader, lr, model_type="LSTM",hidden_dim=24, epochs=1500, criterion = criterion)

In [23]:
outputs, targets, accuracy = evaluate(LSTM_model, fullset_dataloader)

Accuracy: 62.48%


ValueError: too many values to unpack (expected 3)

In [ ]:
outputs

[array([[0.99908066, 0.9980545 , 0.99557966, 0.9295561 ],
        [0.99908066, 0.9980546 , 0.9955798 , 0.9295567 ],
        [0.99908066, 0.99805474, 0.99558   , 0.9295631 ],
        ...,
        [0.9983885 , 0.9691538 , 0.9350392 , 0.21061136],
        [0.9983785 , 0.96876794, 0.93372005, 0.20769623],
        [0.9990809 , 0.9980459 , 0.9955641 , 0.9292424 ]], dtype=float32)]

In [ ]:
targets

[array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        ...,
        [1., 1., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 0., 0.]])]